# Summary:
- Imbalanced sampling (SMOTE) is applied.
- 1D Cov + two layers of LSTM is tested


In [1]:
import pickle

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split
from imblearn.over_sampling import SMOTE, ADASYN

import numpy as np
from keras.models import Model
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform

from keras.callbacks import ModelCheckpoint
from keras.models import Model, load_model, Sequential
from keras.layers import Dense, Activation, Dropout, Input, Masking, TimeDistributed, LSTM, Conv1D
from keras.layers import GRU, Bidirectional, BatchNormalization, Reshape
from keras.optimizers import Adam
np.random.seed(1)

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


# 1. Data Preparation

In [2]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [3]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [4]:
# Use SMOTE oversampling to generate more training data
X_resampled, y_resampled = SMOTE().fit_sample(vectors, data_Y)

In [5]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(X_resampled, y_resampled, test_size=0.25)

In [6]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [7]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((12883, 1024), (12883,), (4295, 1024), (4295,), 10216)

In [8]:
xtr, xt = X_train_array.shape, X_test_array.shape

In [9]:
# reshape dataset
X_train = X_train_array.reshape(xtr[0],32,32) 
Y_train = np.eye(2)[Y_train_array.astype(int)] 
X_test = X_test_array.reshape(xt[0],32,32) 
Y_test = np.eye(2)[Y_test_array.astype(int)] 

# 2. LSTM

In [10]:
def model(input_shape):
    
    X_input = Input(shape = input_shape)
    
    # Cov layer
    X = Conv1D(filters=768, kernel_size=15, strides=4)(X_input)                                 # CONV1D
    X = BatchNormalization()(X)                                 # Batch normalization
    X = Activation('relu')(X)                                 # ReLu activation
    X = Dropout(0.6)(X)                                 # dropout
    
    # LSTM layer 1
    X = LSTM(512, return_sequences=True)(X)
    X = Dropout(0.5)(X)
    
    # LSTM layer 2 
    X = LSTM(512)(X)
    X = Dropout(0.5)(X)
    
    X = Dense(2)(X)  
    # Softmax layer
    X = Activation('softmax')(X)
    
    # Create Model instance
    model = Model(inputs = X_input, outputs = X)
        
    return model

In [11]:
model = model(input_shape = (32,32))

In [12]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32)            0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 5, 768)            369408    
_________________________________________________________________
batch_normalization_1 (Batch (None, 5, 768)            3072      
_________________________________________________________________
activation_1 (Activation)    (None, 5, 768)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 768)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 5, 512)            2623488   
_________________________________________________________________
dropout_2 (Dropout)          (None, 5, 512)            0         
__________

In [13]:
opt = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, decay=0.01)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=["accuracy"])

In [14]:
model.fit(X_train, Y_train, batch_size = 64, epochs=200)

Epoch 1/200
12883/12883 [==============================] - 11s 863us/step - loss: 0.4788 - acc: 0.7770
Epoch 2/200
12883/12883 [==============================] - 8s 619us/step - loss: 0.3884 - acc: 0.8336
Epoch 3/200
12883/12883 [==============================] - ETA: 0s - loss: 0.3525 - acc: 0.850 - 8s 621us/step - loss: 0.3528 - acc: 0.8507
Epoch 4/200
12883/12883 [==============================] - 8s 625us/step - loss: 0.3293 - acc: 0.8608
Epoch 5/200
12883/12883 [==============================] - 8s 623us/step - loss: 0.3064 - acc: 0.8700
Epoch 6/200
12883/12883 [==============================] - 8s 619us/step - loss: 0.2850 - acc: 0.8822
Epoch 7/200
12883/12883 [==============================] - 8s 622us/step - loss: 0.2773 - acc: 0.8863
Epoch 8/200
12883/12883 [==============================] - 8s 620us/step - loss: 0.2583 - acc: 0.8942
Epoch 9/200
12883/12883 [==============================] - 8s 618us/step - loss: 0.2545 - acc: 0.8953
Epoch 10/200
12883/12883 [=================

12883/12883 [==============================] - 8s 606us/step - loss: 0.1239 - acc: 0.9522
Epoch 81/200
12883/12883 [==============================] - 8s 607us/step - loss: 0.1212 - acc: 0.9520
Epoch 82/200
12883/12883 [==============================] - 8s 611us/step - loss: 0.1265 - acc: 0.9482
Epoch 83/200
12883/12883 [==============================] - 8s 618us/step - loss: 0.1243 - acc: 0.9530
Epoch 84/200
12883/12883 [==============================] - 8s 614us/step - loss: 0.1193 - acc: 0.9527
Epoch 85/200
12883/12883 [==============================] - 8s 614us/step - loss: 0.1183 - acc: 0.9545
Epoch 86/200
12883/12883 [==============================] - 8s 614us/step - loss: 0.1159 - acc: 0.9552
Epoch 87/200
12883/12883 [==============================] - 8s 611us/step - loss: 0.1278 - acc: 0.9506
Epoch 88/200
12883/12883 [==============================] - 8s 610us/step - loss: 0.1169 - acc: 0.9537
Epoch 89/200
12883/12883 [==============================] - 8s 613us/step - loss: 0.11

12883/12883 [==============================] - 8s 610us/step - loss: 0.1070 - acc: 0.9581
Epoch 160/200
12883/12883 [==============================] - 8s 606us/step - loss: 0.1029 - acc: 0.9622
Epoch 161/200
12883/12883 [==============================] - 8s 607us/step - loss: 0.0967 - acc: 0.9610
Epoch 162/200
12883/12883 [==============================] - 8s 607us/step - loss: 0.1027 - acc: 0.9615
Epoch 163/200
12883/12883 [==============================] - 8s 607us/step - loss: 0.1019 - acc: 0.9589
Epoch 164/200
12883/12883 [==============================] - 8s 605us/step - loss: 0.0997 - acc: 0.9610
Epoch 165/200
12883/12883 [==============================] - 8s 610us/step - loss: 0.1015 - acc: 0.9608
Epoch 166/200
12883/12883 [==============================] - 8s 606us/step - loss: 0.1019 - acc: 0.9610
Epoch 167/200
12883/12883 [==============================] - 8s 607us/step - loss: 0.0969 - acc: 0.9628
Epoch 168/200
12883/12883 [==============================] - 8s 607us/step - l

In [15]:
loss, acc = model.evaluate(X_test, Y_test)
print("Validation set accuracy = ", acc)

4295/4295 [==============================] - 1s 323us/step
Validation set accuracy =  0.9273573923166473


In [16]:
model_json = model.to_json()
with open ("LSTMModelSMOTE.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("LSTMModelSMOTE.h5")

In [17]:
from sklearn.metrics import classification_report

pred = model.predict(X_test, batch_size=32, verbose=1)
predicted = np.argmax(pred, axis=1)
report = classification_report(np.argmax(Y_test, axis=1), predicted)
print(report)

4295/4295 [==============================] - 1s 284us/step
             precision    recall  f1-score   support

          0       0.93      0.92      0.93      2113
          1       0.93      0.93      0.93      2182

avg / total       0.93      0.93      0.93      4295

